In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [7]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

#每个批次的大小
batch_size = 100
#计算一共多少个批次
n_batch = mnist.train.num_examples//batch_size

#权值初始化
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)#生成一个截断的正态分布
    return tf.Variable(initial)
#初始化偏置
def bias_variable(shape):
    initial =tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#conv layer
def conv2d(x,W):
    # x: input layer Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
    # W:filter [filter_height, filter_width, in_channels, out_channels]
    # strides： Must have `strides[0] = strides[3] = 1`.  For the most common case of the same
    # horizontal and vertices strides, `strides = [1, stride, stride, 1]`. strides[1]:x方向 strides[2]：y方向
    # padding: a string from: SAME, VALID
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#pooling
def max_pool_2x2(x):
    #ksize [1,2,2,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784]) #28*28
y = tf.placeholder(tf.float32,[None,10])

#改变x的格式转为4D的向量[batch,in_height,in_width,in_channels]
x_image = tf.reshape(x,[-1,28,28,1])

#初始化第一个卷积层的权值和偏置值
W_conv1 = weight_variable([5,5,1,32]) #5*5的kernel，32个卷积从1个平面提取特征
b_conv1 = bias_variable([32])#每个kernel一个bias

#把x_image和权值向量进行卷积，再加上偏置值，然后用于relu函数,再池化
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#初始化第二个卷积层的权值和偏置值
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

#第二个卷积层的计算
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#初始化第一个全连接层
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])

#把池化层2的输出转化为偏平的1维
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
#求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

#keep_prob用来表示神经元的输出概率（dropout）
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#初始化第二个权连接层
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
#计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

#cross_entropy
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=predicition))
#使用Adam优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔列表中
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
#求准确率
accuarcy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
            
        acc = sess.run(accuarcy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter" + str(epoch) + ", Testing accuarcy = " +str(acc))
                               


Iter0, Testing accuarcy = 0.9534
Iter1, Testing accuarcy = 0.9722
Iter2, Testing accuarcy = 0.9787
Iter3, Testing accuarcy = 0.9832
Iter4, Testing accuarcy = 0.9848
Iter5, Testing accuarcy = 0.9837
Iter6, Testing accuarcy = 0.9869
Iter7, Testing accuarcy = 0.9877
Iter8, Testing accuarcy = 0.9884


KeyboardInterrupt: 